**Importing Libraries for preprocessing**

In [ ]:
import numpy as np   
import pandas as pd 
import matplotlib.pyplot as plt  
import seaborn as sns
#Library for nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
import string
string.punctuation

In [ ]:
train_df=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.describe()

In [ ]:
#We don't really need keyword , location as a parameter to our predictions
train_df.drop(train_df.iloc[:, 1:3], inplace=True, axis=1)

**Analysing the data**

In [ ]:
train_df.head()

In [ ]:
train_df['target'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
plt.pie(train_df['target'].value_counts(), labels=['Disaster Tweets','Non-Disaster Tweets'])
plt.show()

In [ ]:
train_df['num_characters'] = train_df['text'].apply(len)

In [ ]:
train_df['num_words'] = train_df['text'].apply(lambda x:len(nltk.word_tokenize(x)))

In [ ]:
train_df['num_sentences'] = train_df['text'].apply(lambda x:len(nltk.sent_tokenize(x)))

In [ ]:
train_df.head()

In [ ]:
plt.figure(figsize=(12,5))
sns.histplot(train_df[train_df['target'] == 0]['num_characters'])
sns.histplot(train_df[train_df['target'] == 1]['num_characters'],color='red')
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
sns.histplot(train_df[train_df['target'] == 0]['num_words'])
sns.histplot(train_df[train_df['target'] == 1]['num_words'],color='red')
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
sns.histplot(train_df[train_df['target'] == 0]['num_sentences'])
sns.histplot(train_df[train_df['target'] == 1]['num_sentences'],color='red')
plt.show()


**Preprocessing Texts**

In [ ]:
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
    text = y[:]
    y.clear()
    
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    text = y[:]
    y.clear()
    for i in text:
        y.append(ps.stem(i))
    return " ".join(y)    

In [ ]:
#testing our preprocess
transform_text("Several people have perished from the earthquake till now.")

In [ ]:
train_df['transformed text'] = train_df['text'].apply(transform_text)

In [ ]:
train_df.head()

**Defining the model**

In [ ]:
#Importing text vectorizers , the most commonly used Count and TFidf
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
cv = CountVectorizer()
tfidf = TfidfVectorizer()

In [ ]:
X = tfidf.fit_transform(train_df['transformed text']).toarray()

In [ ]:
X.shape

In [ ]:
X

In [ ]:
y = train_df['target'].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [ ]:
from sklearn.naive_bayes import GaussianNB , MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [ ]:
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [ ]:
Nb_models = [gnb,mnb,bnb]

In [ ]:
for model in Nb_models:
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print(model)
    print("\nAccuracy Score:\n")
    print(accuracy_score(y_test,y_pred))
    print("\nConfusion Matrix:\n")
    print(confusion_matrix(y_test,y_pred))
    print("\nPrecision Score:\n")
    print(precision_score(y_test,y_pred))
    print("\n********************************************\n")

In [ ]:
# We can see above that Gaussian Naive Bayes has yielded underwhelming result
# Next plan of action would be to try the most popularly used Scikit-Learn Classifiers and compare the results to end up with a single model at end.
# Parameters to consider: Accuracy , Precison , Performance and Processing Time

In [ ]:
#Importing classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [ ]:
svc = SVC(kernel='sigmoid')
knc = KNeighborsClassifier()
mnb = MultinomialNB()
bnb = BernoulliNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression()
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)
xgb = XGBClassifier(n_estimators=50,random_state=2)

In [ ]:
clfs = {
    'SVC' : svc,
    'KN' : knc, 
    'NB': mnb,
    'NB1':bnb,
    'DT': dtc, 
    'LR': lrc, 
    'RF': rfc, 
    'AdaBoost': abc, 
    'BgC': bc, 
    'ETC': etc,
    'GBDT':gbdt,
    'xgb':xgb
}

In [ ]:
def train_classifier(clf,X_train,y_train,X_test,y_test):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    
    return accuracy,precision

In [ ]:
accuracy_scores = []
precision_scores = []

for name,clf in clfs.items():
    
    current_accuracy,current_precision = train_classifier(clf, X_train,y_train,X_test,y_test)
    
    print("For ",name)
    print("Accuracy - ",current_accuracy)
    print("Precision - ",current_precision)
    
    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)

In [ ]:
performance_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy':accuracy_scores,'Precision':precision_scores}).sort_values('Precision',ascending=False)

In [ ]:
performance_df

**Final Model Selection**

In [ ]:
# Selecting the best results yet
final_models = {'lr': LogisticRegression(),
                'mnb': MultinomialNB(),
                'bnb' :BernoulliNB()} 

In [ ]:
import time

def classify(clf, X_train, X_test, y_train, y_test):
    y_pred = clf.fit(X_train, y_train).predict(X_test)
    precision = precision_score(y_test, y_pred)
    train_accuracy = clf.score(X_train, y_train)
    test_accuracy = accuracy_score(y_test, y_pred)
    return precision, train_accuracy, test_accuracy

for name, clf in final_models.items():
    i_precision, i_train_accuracy, i_test_accuracy = classify(clf, X_train, X_test, y_train, y_test)
    
    # check the time
    start = time.time()
    
    # fit and predict
    y_pred = clf.fit(X_train, y_train).predict(X_test)
    
    # check the time
    end = time.time()
    
    # print out the scores and classification reports
    print('[{}]\nProcessing Time: {} secs | Test Accuracy: {} | Precision: {}\n'.format(name, 
                                                                                            round(end-start, 2),
                                                                                            round(i_test_accuracy, 2),
                                                                                            round(i_precision, 2)))

In [ ]:
test_df=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
test_df.head()

In [ ]:
test_df['transformed_text'] = test_df['text'].apply(transform_text)

In [ ]:
eval=tfidf.transform(test_df['transformed_text']).toarray()

In [ ]:
start = time.time()
final_pred =bnb.predict(eval)
end = time.time()
pred_time = end - start

print('Prediction time: {} secs'.format(round(pred_time, 2)))

In [ ]:
submission = test_df[['id']].reset_index(drop=True)
submission['target'] = final_pred.astype('int64')
submission

In [ ]:
submission.to_csv('submission.csv', index=False)